## Seria Lab: add a new ship
This is a legacy demostration with obsolete seria API which previously built upon `treelib` library.  
Experiment and understand the structure of the seria file. Demonstrate the use of seria module to parse and modifying the seria file

In [1]:
import seria

profile_tree = seria.load('sample/profile.seria')

# design_ship_tree = seria.load('sample/Courageous.seria')
# design_ship_tree = seria.load('sample/Meteor Mk2.seria')
design_ship_tree = seria.load('sample/Rook.seria')

### Add a new ship to an existing fleet

A ship in the profile has exactly the same structure (nodes) compare to a ship design file. This means we can easily copy the data from a design file and paste it into the profile. However, you need to change relavent id and add additional ids, in order to make it working correctly. Failing to do so will cause the game to crash.

In [2]:
# m_id are unique
ids = list(map(int, profile_tree.get_children_attribute('m_id')))

unique_ids = set(ids)

print('id count:', len(ids))
print('unique id count:', len(unique_ids))

id count: 2959
unique id count: 2959


In [3]:
def attribute_diff(a, b):
    return set(a) - set(b)


# attribute difference between in-profile ship and ship design
p_fleet_tree = profile_tree.get_subtree_if(
    lambda node: node.get_attribute('m_name') == 'MARK')

# assume having the ship Sevastopol
p_ship_tree = p_fleet_tree.get_subtree_if(
    lambda node: node.get_attribute('m_name') == 'Rook')

print('difference of root Node node:')
p_ship_attribute_names = p_ship_tree.get_attribute_names()
design_ship_attribute_names = design_ship_tree.get_attribute_names()
for name in sorted(attribute_diff(p_ship_attribute_names, design_ship_attribute_names)):
    print(' ', name)

print('difference of Creature node:')
p_creature_tree = p_ship_tree.get_subtree_by_class(
    'Frame').get_subtree_by_class('Body').get_subtree_by_class('Creature')
design_creature_tree = design_ship_tree.get_subtree_by_class(
    'Frame').get_subtree_by_class('Body').get_subtree_by_class('Creature')
seva_creature_attribute_names = p_creature_tree.get_attribute_names()
cour_creature_attribute_names = design_creature_tree.get_attribute_names()
for name in sorted(attribute_diff(seva_creature_attribute_names, cour_creature_attribute_names)):
    print(' ', name)

difference of root Node node:
  m_master_id
  m_state
difference of Creature node:
  m_alignment
  m_bio_snapshot
  m_escadra.id
  m_escadra_index
  m_moral
  m_rad_seekness_timer
  m_radiation_extra
  m_rest
  m_tarkhan
  m_tele_crew_total
  shipLoadTime
  wasAddedToPlayerEscadra


By looking at the result above, we know that some attribute is required for a ship in a fleet:
- `m_master_id`
- `m_state`
- `m_alignment`
- `m_escadra.id`
- `m_escadra_index`
- `m_moral`
- `m_tarkhan`
- `m_tele_crew_total`
- `wasAddedToPlayerEscadra`

Additionally, `creatureId` is also needed to be set for the Creature node inside the ship. `nextCreatureId` give the right value for the next one and will also be updated.

Steps to config a new ship node:
1. set `m_master_id` of the root Node node to its target fleet `m_id` 
2. generate a new `m_id` and set to the root node and update relevant id reference to new `m_id`
3. generate new `m_id` for each Body node and update relevant id
4. generate a new `m_id` for the Frame node and update relevant id
5. generate a new `m_id` for the Creature node and update relevant id

In [ ]:
# make new id unique from existing ids
# next_id = max(unique_ids) + 1

# obtain correct attribute values from profile
next_creature_id = profile_tree.get_attribute('nextCreatureId')

p_fleet_tree = profile_tree.get_subtree_if(
    lambda node: node.get_attribute('m_name') == 'MARK')

alignment = p_fleet_tree.get_attribute('m_alignment')
escadra_id = p_fleet_tree.get_attribute('m_id')

# make new escadra index for the new ship
escadra_indexes = set()
for node in p_fleet_tree.all_nodes_itr():
    if node.tag == 'Creature':
        escadra_indexes.add(int(node.get_attribute('m_escadra_index')))

design_ship_tree.set_attribute('_header', 'm_children=7')
design_ship_tree.set_attribute('m_master_id', escadra_id)

# update root Node node
# design_ship_tree.update_attribute('m_id', str(next_id))
design_ship_tree.set_attribute('m_state', '2')

# update all depth 2 Body nodes
# for tree in design_ship_tree.get_subtrees_by_class('Body'):
#     next_id += 1
#     previous_id = tree.get_attribute('m_id')
#     tree.update_attribute('m_id', str(next_id))
#     # update Joint nodes and Slot nodes
#     for node in design_ship_tree.all_nodes_itr():
#         node.update_attribute_by_value(previous_id, str(next_id))

# update Frame node
design_frame_tree = design_ship_tree.get_subtree_by_class('Frame')

# next_id += 1
# design_frame_tree.update_attribute('m_id', str(next_id))

# update all child Body nodes in Frame node
# for tree in design_frame_tree.get_subtrees_by_class('Body'):
#     next_id += 1
#     previous_id = tree.get_attribute('m_id')
#     tree.update_attribute('m_id', str(next_id))
#     for node in design_ship_tree.all_nodes_itr():
#         node.update_attribute_by_value(previous_id, str(next_id))

# update Creature node
design_creature_tree = design_frame_tree.get_subtree_by_class('Body').get_subtree_by_class(
    'Creature')

# next_id += 1
# design_creature_tree.update_attribute('m_id', str(next_id))

design_creature_tree.set_attribute('creatureId', next_creature_id)
profile_tree.set_attribute('nextCreatureId', str(int(next_creature_id) + 1))

design_creature_tree.set_attribute('m_alignment', alignment)
design_creature_tree.set_attribute('m_escadra.id', escadra_id)
design_creature_tree.set_attribute(
    'm_escadra.index', str(max(escadra_indexes) + 1))
design_creature_tree.set_attribute('m_moral', '10')
design_creature_tree.set_attribute('m_tarkhan', 'DAUD')

crew_capacity = design_creature_tree.get_attribute('m_tele_crew_capacity')
design_creature_tree.set_attribute('m_tele_crew_total', crew_capacity)
design_creature_tree.set_attribute('m_tele_crew_capacity', crew_capacity)

# design_creature_tree.set_attribute('shipLoadTime', '0')
design_creature_tree.set_attribute('wasAddedToPlayerEscadra', 'true')

In [34]:
# copy the ship tree to the profile

profile_tree.paste(p_fleet_tree.root, design_ship_tree)

file = open('profile.seria', 'w', encoding='cp1251')
file.write(seria.dump(profile_tree))

file.close()